## From corpus to noun term frequency sparse matrix

In [1]:
from pprint import pprint
from konlpy.tag import Twitter

In [2]:
twitter = Twitter()

In [3]:
twitter.nouns("이게 바로 테스트라는 녀석")

['이', '바로', '테스트', '녀석']

In [4]:
preprocessed_data_fname = './data/small_naver_news/processed/corpus.txt'
model_folder = './tmp/'

In [5]:
with open(preprocessed_data_fname, encoding="utf-8") as f:
    docs = [doc.strip() for doc in f]

In [6]:
docs[:2]

["(의왕 국회사진기자단=연합뉴스) 김성태 '최순실 국조특위' 위원장이 26일 오전 경기도 의왕시 서울구치소에서 열린 현장청문회에 입장하고 있다. 2016.12.26  scoop@yna.co.kr",
 "(서울=연합뉴스) 이재희 기자 = 소녀시대의 태연이 26일 오후 서울 강남구 코엑스에서 열린 'SBS 어워즈 페스티벌(SAF) 가요대전'에서 포즈를 취하고 있다. 2016.12.26  scape@yna.co.kr"]

In [7]:
from collections import Counter

In [8]:
%%time
noun_counter = Counter([noun for doc in docs for noun in twitter.nouns(doc)])
print(len(noun_counter))

12612
CPU times: user 18.6 s, sys: 433 ms, total: 19 s
Wall time: 15.5 s


In [9]:
noun_counter

Counter({'프리': 7,
         '오송역': 2,
         '논산시': 2,
         '수원': 12,
         '멕시코시티': 3,
         '완패': 1,
         '테이크': 1,
         '국악인': 1,
         '무한도전': 1,
         '내달': 16,
         '게임기': 1,
         '김현미': 1,
         '기숙': 1,
         '최태민': 5,
         '청렴': 11,
         '열람': 4,
         '신경질': 2,
         '중대': 3,
         '노선': 39,
         '가상': 5,
         '신한은행': 1,
         '노부': 1,
         '김윤정': 1,
         '악화': 10,
         '붕괴': 4,
         '사려': 2,
         '동래': 1,
         '그래미': 2,
         '지청': 5,
         '산업화': 2,
         '김대성': 1,
         '속삭임': 1,
         '뿌리': 6,
         '운영자': 1,
         '로그아웃': 4,
         '격인': 5,
         '한류': 6,
         '연보': 4,
         '이필우': 1,
         '단정': 2,
         '도용': 1,
         '운': 2,
         '이도': 3,
         '악명': 2,
         '연옥': 1,
         '소문': 3,
         '부서': 18,
         '학': 20,
         '극단': 9,
         '반대': 31,
         '방식': 53,
         '노동': 8,
         '산화물': 1,
         '석창':

In [10]:
for min_count in [2,3,5,10]:
    _counter = {word for word ,freq in noun_counter.items() if freq >= min_count}
    print("num of noun (min_count = {}): {:>10}".format(min_count, len(_counter)))

num of noun (min_count = 2):       7772
num of noun (min_count = 3):       5900
num of noun (min_count = 5):       4102
num of noun (min_count = 10):       2424


In [11]:
noun_dict = {word:freq for word, freq in noun_counter.items() if freq >= 5}

def custom_tokenizer(doc):
    return [word for word in twitter.nouns(doc) if word in noun_dict]

print(twitter.nouns(docs[1]))
print(custom_tokenizer(docs[1]))

['서울', '연합뉴스', '이재희', '기자', '소녀시대', '태연', '일', '오후', '서울', '강남구', '코엑스', '어워즈', '페스티벌', '가요대전', '에서', '포즈']
['서울', '연합뉴스', '이재희', '기자', '일', '오후', '서울', '강남구', '코엑스', '어워즈', '페스티벌', '가요대전', '에서', '포즈']


# CountVectorizer(sklearn)

In [12]:
from sklearn.feature_extraction.text import CountVectorizer

In [13]:
vectorizer = CountVectorizer(tokenizer=custom_tokenizer, min_df=0.05, max_df=0.95)
x_sparse = vectorizer.fit_transform(docs)

In [14]:
x_sparse

<1355x165 sparse matrix of type '<class 'numpy.int64'>'
	with 24394 stored elements in Compressed Sparse Row format>

In [15]:
vocab2int = vectorizer.vocabulary_
vocab2int["가운데"]

0

In [16]:
int2vocab = [word for word, index in sorted(vocab2int.items(), key=lambda x:x[1])]

In [17]:
int2vocab[0]

'가운데'

# Sparse matrix  VS  dense matrix

In [18]:
from scipy.io import mmread, mmwrite

In [19]:
mmwrite("./x.mm", x_sparse)

In [20]:
x_sparse.data.shape

(24394,)

In [21]:
x_dense = x_sparse.todense()

In [22]:
x_dense.shape

(1355, 165)

In [23]:
x_dense

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 1, ..., 0, 0, 0],
        [0, 1, 0, ..., 2, 0, 0],
        ..., 
        [0, 0, 1, ..., 0, 0, 0],
        [1, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

# vector

In [24]:
import numpy as np

In [25]:
x_tmp = np.random.random((5,10))

In [26]:
x_tmp

array([[ 0.95315614,  0.5082742 ,  0.28877455,  0.10664321,  0.9263194 ,
         0.91827904,  0.11910329,  0.27432708,  0.31919597,  0.4163751 ],
       [ 0.00633287,  0.886484  ,  0.34393892,  0.81403794,  0.93178979,
         0.86204803,  0.06362372,  0.33823388,  0.11930462,  0.15228484],
       [ 0.16879788,  0.54125412,  0.48345942,  0.24713418,  0.37422912,
         0.71345402,  0.28354914,  0.29175758,  0.23597413,  0.47917505],
       [ 0.4911982 ,  0.49561553,  0.02805547,  0.26174725,  0.024387  ,
         0.39666543,  0.52494993,  0.69413965,  0.27120681,  0.12620528],
       [ 0.41900382,  0.32045178,  0.44863997,  0.20037298,  0.85556027,
         0.7211033 ,  0.28706062,  0.44252629,  0.8078919 ,  0.93483613]])

In [27]:
# row sum
x_tmp.sum(axis=0)

array([ 2.03848892,  2.75207962,  1.59286833,  1.62993556,  3.11228558,
        3.61154981,  1.2782867 ,  2.04098449,  1.75357343,  2.10887639])

In [28]:
# column
x_tmp.sum(axis=1)

array([ 4.83044797,  4.51807862,  3.81878463,  3.31417055,  5.43744707])

In [29]:
# word frequency (1doc = row)
x_tmp.sum(axis=0) / x_tmp.sum()

array([ 0.0930013 ,  0.12555721,  0.0726709 ,  0.07436201,  0.14199077,
        0.16476854,  0.05831885,  0.09311516,  0.0800027 ,  0.09621257])

In [30]:
x1 = np.random.random((1,10))
x2 = np.random.random((1,10))

score = x1 / (x1 + x2)

In [31]:
score

array([[ 0.62473783,  0.52319967,  0.57213033,  0.39763918,  0.43597207,
         0.70560492,  0.90913452,  0.55516312,  0.31570881,  0.27231098]])

In [32]:
def keyword_extraction(x_dense, aspect_word_index, int2vocab, threshold=0.7):
    dt_idx = []
    dr_idx = []
    
    # 모든 문서에서 aspect_word의 column 가져오기
    for idx, tf in enumerate(x_dense[:, aspect_word_index]):
        # tf 가 0보다 크면 target (positive) document set, 
        if tf > 0:
            dt_idx.append(idx)
        # tf == 0은 aspect word가 등장하지 않았으므로 reference document set
        else:
            dr_idx.append(idx)
    
    # x_dense에 list로 이뤄진 dt_idx를 넣으면, 해당 index의 rows만을 추려서 submatrix를 만들어줍니다. 
    x_dt = x_dense[dt_idx]
    x_dr = x_dense[dr_idx]
    
    # term proportion matrix를 만듭니다. 
    x_dt = x_dt.sum(axis=0) / x_dt.sum()
    x_dr = x_dr.sum(axis=0) / x_dr.sum()
    
    # scure를 구한 뒤 reshape을 이용하여 column vector 형태로 만들어줍니다. 
    x_score = x_dt / (x_dt + x_dr)
    x_score = np.asarray(x_score).reshape(-1)
    
    # score가 threshold 이상인 단어들의 index를 keywords list에 넣어둡니다. 
    keywords = [(word_idx, score) for word_idx, score in enumerate(x_score)]
    
    # index2word를 이용하여 word index를 키워드로 바꿔줍니다. 
    keywords = [(int2vocab[word_idx], score) for word_idx, score in keywords]
    
    return keywords

In [33]:
for word in ['연합뉴스', '대통령', '최순실']:
    
    if not word in vocab2int:
        continue
        
    print('\n\nAspect word = %s (%d)' % (word, vocab2int[word]))
    
    word_index = vocab2int[word]
    keywords_ = keyword_extraction(x_dense, word_index, int2vocab, threshold=0.7)
    
    # 길이가 1인 단어들 제거
    keywords = [(word, score) for word, score in keywords_ if len(word) > 1]
    
    # 추출된 키워드를 term frequency 기준으로 정렬하여 상위 20개 선택
    keywords_tops = sorted(keywords, key=lambda x:noun_counter.get(x[0], 0), reverse=True)[:20]
    
    for keyword, score in keywords_tops:
        print('%10s\t%.3f' % (keyword, score))



Aspect word = 연합뉴스 (97)
      연합뉴스	1.000
        서울	0.822
        기자	0.914
        오전	0.815
       대통령	0.419
        의원	0.419
        정부	0.790
        대표	0.644
       최순실	0.215
        억원	0.762
        사업	1.000
        의혹	0.223
        수사	0.242
        지역	0.743
        오후	0.416
        내년	0.572
        지난	0.571
        사진	0.303
        국회	0.466
        대한	0.723


Aspect word = 대통령 (35)
      연합뉴스	0.315
        서울	0.324
        기자	0.321
        오전	0.265
       대통령	1.000
        의원	0.565
        정부	0.577
        대표	0.513
       최순실	0.602
        억원	0.113
        사업	0.224
        의혹	0.827
        수사	0.775
        지역	0.251
        오후	0.207
        내년	0.304
        지난	0.492
        사진	0.434
        국회	0.453
        대한	0.584


Aspect word = 최순실 (150)
      연합뉴스	0.344
        서울	0.397
        기자	0.365
        오전	0.524
       대통령	0.795
        의원	0.644
        정부	0.592
        대표	0.168
       최순실	1.000
        억원	0.056
        사업	0.162
        의혹	0.918
        수사	0.934
        지역	0.137
     

In [34]:
# TODO: Aspect word 없이 문서에서 키워드 추출하는 법